# Learning Journey AIG03
## Leverage Document Grounding in Orchestration Service for RAG-based Content Generation

In this learning journey, you will learn how to leverage the Document Grounding module in the Orchestration Service to generate content using the Retrieval-Augmented Generation (RAG) approach.
The Document Grounding module helps in grounding the input questions to relevant documents.
The grounding process involves retrieving relevant documents from a knowledge base and using them to high-quality generate responses.
The knowledge base can be a collection of documents in a sharepoint folder, an elastic search engine, or data repository which contains vectors.

In this learning journey, you will perform the following steps:
- Create the knowledge base with the relevant documents.
- Configure the Document Grounding module in the Orchestration Service.
- Generate content based on the knowledge base using the RAG approach.


## Prerequisites
Install the Generative AI Hub SDK using the following command:

In [ ]:
%pip install generative-ai-hub-sdk

How to set the credentials for the Generative AI Hub SDK:
see the AICore documentation for available options. [Note: maybe show or recommend an option here like local config file or environment variables]



#### Todo: Provide credentials for Sid.


#### Step 1: Create a Vector Knowledge Base
   - Upload a set of documents which will automatically be vectorized and added to the knowledge base.

Thera are 2 options to create the Knowledge Base:
- Upload the documents to a sharepoint folder and run Pipeline API to vectorize the documents. (This option includes more detailed steps, which we have not yet performed. We still miss some information here.)
- Upload the documents via AI Data Management (https://help.sap.com/docs/sap-ai-core/generative-ai-hub/create-resource-group-for-ai-data-management?locale=en-US)

The AI Core Help describes in section Data API how to upload Documents
https://help.sap.com/docs/sap-ai-core/generative-ai-hub/vector-api?locale=en-US

For 1st version of the Learning Journey we should just reference the APIs and describe the sequence. We assume the user can create the Vector store herself.

In 2nd version of the Learning Journey we add the sharepoint option. Maybe there is better documentation available for both options until then, which we can reference.

#### Step 2: Configure the Document Grounding Module
   - Define the configuration for the Document Grounding module in the Orchestration Service.

### Add the url to the orchestration service here
Todo: guide the user to get the url from the AI Core service / AI Launchpad

In [1]:
orchestration_service_url = "https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dedccdc403fb2b8a"
#"https://api.ai.prodeuonly.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/df6aaf7f157a488e"
#"https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dedccdc403fb2b8a"
# 'https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d4394c82de511a6a'


In [3]:
# Import libraries
from gen_ai_hub.proxy import get_proxy_client
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.document_grounding import (GroundingModule, DocumentGrounding, GroundingFilterSearch,
                                                                DataRepositoryType, DocumentGroundingFilter)
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService

In [4]:
# Set up the Orchestration Service
aicore_client = get_proxy_client().ai_core_client
orchestration_service = OrchestrationService(api_url=orchestration_service_url)
llm = LLM(
    name="gpt-4o",
    parameters={
        'temperature': 0.0,
    }
)
template = Template(
            messages=[
                SystemMessage("""Facility Solutions Company provides services to luxury residential complexes, apartments,
                individual homes, and commercial properties such as office buildings, retail spaces, industrial facilities, and educational institutions.
                Customers are encouraged to reach out with maintenance requests, service deficiencies, follow-ups, or any issues they need by email.
                """),
                UserMessage("""You are a helpful assistant for any queries for answering questions.
                Answer the request by providing relevant answers that fit to the request.
                Request: {{ ?user_query }}
                Context:{{ ?grounding_response }}
                """),
            ]
        )

In [16]:
# Set up Document Grounding
filters = [DocumentGroundingFilter(id="vector",
                                   data_repositories=["3068256c-5f92-4079-8667-6caf0037fd01"],  #"92014546-44a7-4770-af7f-ba6628aae09f"],
                                   search_config=GroundingFilterSearch(max_chunk_count=15),
                                   data_repository_type=DataRepositoryType.VECTOR.value
                                   )
]

grounding_config = GroundingModule(
            type="document_grounding_service",
            config=DocumentGrounding(input_params=["user_query"], output_param="grounding_response", filters=filters)
        )

config = OrchestrationConfig(
    template=template,
    llm=llm,
    grounding=grounding_config
)

 #### Step 3: Generate context-relevant answer for a user query
   - Run the Orchestration Service with the Configuration from the previous step.

In [18]:
response = orchestration_service.run(config=config,
                            template_values=[
                                TemplateValue("user_query", "List the issues that are reported by customers."),
                            ])
print(response.orchestration_result.choices[0].message.content)

Based on the provided context, here is a list of issues reported by customers:

1. **HVAC System Noise** - Minor noise issue after repair at Lakeview Corporate Offices.
2. **Landscaping Service Issues** - Improperly trimmed shrubs and debris left behind at Crestview Gardens Apartments.
3. **Window Cleaning Oversight** - Missed windows in the east wing of Riverfront Business Complex.
4. **AC Malfunction** - Malfunctioning air conditioning unit at Sunridge Mall.
5. **Janitorial Service Oversight** - Some classrooms skipped during cleaning at Brookdale High School.
6. **Elevator Malfunction** - Malfunctioning elevator in Building B at Oakwood Corporate Center.
7. **Heating System Malfunction** - Urgent heating system issue at Greenview Residences.
8. **Electrical Issues** - Flickering lighting in the showroom at Midtown Motors.
9. **Plumbing Leak** - Persistent leak and water pressure issue in the kitchen area at Skyview Towers.
10. **Water Damage** - Water damage on the ceiling of Unit 4

In [19]:
print(response.module_results.grounding.data['grounding_result'])


Subject: Feedback on HVAC Repair

Hello,

I wanted to thank your team for the prompt repair of our HVAC system at Lakeview Corporate Offices. However, there’s been a minor noise issue since. Is it possible to have a technician look into this?

Warm regards,
Robert Kim```Subject: Complaint About Recent Landscaping Service

Dear Facility Solutions Team,

I’m writing to report some issues with the recent landscaping service at Crestview Gardens Apartments. The shrubs were not trimmed properly, and debris was left behind. Can someone come to address this soon?

Sincerely,
James Anderson```Subject: Complaint: Window Cleaning Service Oversight

Dear Facility Solutions,

I wanted to bring to your attention that the windows in the east wing of the Riverfront Business Complex were missed during the last cleaning service. This has been recurring, and I would appreciate it if we could resolve this soon.

Regards,
Mark Phillips```Subject: Urgent Request for AC Repair

Facility Solutions Team,

The